In [2]:
import sagemaker
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import boto3
import re
from sagemaker.amazon.amazon_estimator import get_image_uri
import os
import urllib.request
import json
import logging
import shutil
from imageio import imread
import scipy.io as sio
import warnings
warnings.filterwarnings('ignore')

In [3]:
%%bash
mkdir car-generated car-train car-train_annotation car-validation car-validation_annotation

In [4]:
def download_data(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

In [5]:
download_data('http://imagenet.stanford.edu/internal/car196/cars_train.tgz')

In [7]:
download_data('http://ai.stanford.edu/~jkrause/cars/car_devkit.tgz')

In [8]:
%%bash
tar -xzf car_devkit.tgz
tar -xzf cars_train.tgz

In [10]:
bucket = 'awsmlprac'
prefix = 'ObjectDetection'

role = get_execution_role()
sess = sagemaker.Session()

In [14]:
training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version = 'latest')

In [21]:
def getClasses(matFile):
    content = sio.loadmat(matFile)
    classes = [(_[0]) for _ in content['class_names'][0]]
    return classes

def getAnnos(matFile):
    content = sio.loadmat(matFile)
    return content['annotations'][0]

def get_mapper_fn(map):
    def mapper(in_category):
        return map[in_category]
    return mapper

def get_class_mapper():
    original_list = [i for i in range(197)][1:]
    iter_counter = 0
    CLASS = {}
    for orig in original_list:
        CLASS[orig] = iter_counter
        iter_counter += 1
    return CLASS

In [16]:
fix_index_mapping = get_mapper_fn(get_class_mapper())

In [17]:
fix_index_mapping

<function __main__.get_mapper_fn.<locals>.mapper(in_category)>

In [18]:
images = os.listdir('cars_train')
for f in images[:]:
    if not(f.endswith(".jpg")):
        images.remove(f)

In [19]:
images[0:5]

['02963.jpg', '05003.jpg', '04527.jpg', '00463.jpg', '01302.jpg']

In [22]:
categories = getClasses('devkit/cars_meta.mat')
annotations = getAnnos('devkit/cars_train_annos.mat')

In [23]:
print(len(categories))
print(len(annotations))

196
8144


In [25]:
for img in images:
    shape = imread('cars_train/{}'.format(img)).shape
    jsonFile = img.split('.')[0]+'.json'
    
    line = {}
    line['file'] = img
    line['image_size'] = [{'width': int(shape[1]),
                          'height': int(shape[0]),
                          'depth': 3}]
    line['annotations'] = []
    line['categories'] = []
    #print annotations
    for anno in annotations:
        if(anno[5][0] == img):
            line['annotations'].append({'class_id': int(fix_index_mapping(anno[4][0][0])),
                                       'top': int(anno[1][0][0]),
                                       'left': int(anno[0][0][0]),
                                       'width': abs(int(anno[2][0][0]) - int(anno[0][0][0])),
                                       'height': abs(int(anno[3][0][0]) - int(anno[1][0][0]))})
            class_name = ''
            for ind, cat in enumerate(categories, start = 1):
                if int(anno[4][0][0]) == ind:
                    class_name = str(cat)
            assert class_name is not ''
            line['categories'].append({'class_id': int(anno[4][0][0]),
                                      'name': class_name})
    if line['annotations']:
        with open(os.path.join('car-generated', jsonFile), 'w') as p:
            json.dump(line, p)

In [26]:
#sample json entry
line

{'file': '06881.jpg',
 'image_size': [{'width': 640, 'height': 360, 'depth': 3}],
 'annotations': [{'class_id': 83,
   'top': 75,
   'left': 28,
   'width': 590,
   'height': 215}],
 'categories': [{'class_id': 84, 'name': 'Dodge Caliber Wagon 2007'}]}

In [27]:
jsons = os.listdir('car-generated')
print('{} images with annotation'.format(len(jsons)))

8144 images with annotation


In [30]:
train_jsons = jsons[:6516]  #80% data
val_jsons = jsons[6516:]  #20% data

#sorting data in train and validation folders to have separate data channels
for i in train_jsons:
    image_file = './cars_train/'+i.split('.')[0]+'.jpg'
    if os.path.exists(image_file):
        shutil.move(image_file, './car-train/')
        shutil.move('./car-generated/'+i,'./car-train_annotation/')

for i in val_jsons:
    image_file = './cars_train/'+i.split('.')[0]+'.jpg'
    if os.path.exists(image_file):
        shutil.move(image_file, './car-validation/')
        shutil.move('./car-generated/'+i, './car-validation_annotation/')

In [31]:
train_channel = prefix+'/car-train'
validation_channel = prefix+'/car-validation'
train_annotation_channel = prefix+'/train_annotation'
validation_annotation_channel = prefix+'/validation_annotation'

sess.upload_data(path = 'car-train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path = 'car-validation', bucket=bucket, key_prefix=validation_channel)
sess.upload_data(path = 'car-train_annotation', bucket=bucket, key_prefix=train_annotation_channel)
sess.upload_data(path = 'car-validation_annotation', bucket = bucket, key_prefix=validation_annotation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucket, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucket, validation_annotation_channel)

In [32]:
#output location to save the model
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [33]:
#build the model
estimator = sagemaker.estimator.Estimator(training_image,
                                         role,
                                         train_instance_count = 1,
                                         train_instance_type = 'ml.p2.8xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode = 'File',
                                         output_path = s3_output_location,
                                         sagemaker_session = sess)

In [34]:
#tuning the hyper parameter
estimator.set_hyperparameters(base_network = 'vgg-16',
                             use_pretrained_model = 1,
                             num_classes = 196,
                             mini_batch_size = 16,
                             epochs = 10,
                             learning_rate = 0.001,
                             lr_scheduler_step = '10',
                             lr_scheduler_factor = 0.1,
                             optimizer = 'sgd',
                             momentum = 0.9,
                             weight_decay = 0.0005,
                             overlap_threshold = 0.5,
                             nms_threshold = 0.45,
                             image_shape = 512,
                             label_width = 600,
                             num_training_samples = 6516)

In [37]:
train_data = sagemaker.session.s3_input(s3_train_data,
                                       distribution = 'FullyReplicated',
                                       content_type = 'image/jpeg',
                                       s3_data_type = 'S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data,
                                       distribution = 'FullyReplicated',
                                       content_type = 'image/jpeg',
                                       s3_data_type = 'S3Prefix')
train_annotation = sagemaker.session.s3_input(s3_train_annotation,
                                             distribution = 'FullyReplicated',
                                             content_type = 'image/jpeg',
                                             s3_data_type = 'S3Prefix')
validation_annotation = sagemaker.session.s3_input(s3_validation_annotation,
                                             distribution = 'FullyReplicated',
                                             content_type = 'image/jpeg',
                                             s3_data_type = 'S3Prefix')

In [38]:
data_channels = {'train': train_data, 'validation':validation_data,
                'train_annotation':train_annotation, 'validation_annotation':validation_annotation}
data_channels

{'train': <sagemaker.inputs.s3_input at 0x7f35bb8104e0>,
 'validation': <sagemaker.inputs.s3_input at 0x7f35bb8103c8>,
 'train_annotation': <sagemaker.inputs.s3_input at 0x7f35ba77b3c8>,
 'validation_annotation': <sagemaker.inputs.s3_input at 0x7f35bb810128>}

In [39]:
#fitting the model
estimator.fit(inputs = data_channels, logs=True)

In [40]:
#deploying model
object_detector = estimator.deploy(initial_instance_count=1,
                                  instance_type = 'ml.m4.xlarge')

In [41]:
#file_name = 'photo.jpg'
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)

In [42]:
import json
object_detector.content_type = 'image/jpeg'
results = object_detector.predict(b)
detections = json.loads(results)

In [43]:
len(detections['prediction'])

In [ ]:
#code provided by AWS for comprehending detection from trained oobject detection model
def visualize_detection(img_file, dets, classes = [], thresh=0.6):
    import random
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    
    img = mpimg.imread(img_file)
    plt.imshow(img)
    height = img.shape[0]
    width = img.shape[1]
    colors = dict()
    for det in dets:
        (klass, score, x0, y0, x1, y1) = det
        if score < thresh:
            continue
        cls_id = int(klass)
        if cls_id not in colors:
            colors[cls_id] = (random.random(),random.random(),random.random())
        xmin = int(x0 * width)
        ymin = int(y0 * height)
        xmax = int(x1 * width)
        ymax = int(y1 * height)
        rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                            ymax - ymin, fill = False,
                            edgecolor = colors[cls_id],
                            linewidth = 3.5)
        plt.gca().add_patch(rect)
        class_name = str(cls_id)
        if classes and len(classes) > cls_id:
            class_name = classes[cls_id]
            print(class_name)
        plt.gca().text(xmin, ymin-2,
                      '{:s} {:.3f}'.format(class_name, score),
                      bbox = dict(facecolor = colors[cls_id], alpha=0.5), fontsize = 12, color = 'red')
        plt.show()